# Puzzle Input

## Sample Input

In [1]:
val sampleInput = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
""".trimIndent()

## Actual Input

In [2]:
val actualInput = """
...
""".trimIndent()

# Part 1

In [3]:
data class Entity(val type: String, val id: Long)
data class RangeMapping(val srcRange: LongRange, val offset: Long)
data class TypeMapping(val srcType: String, val destType: String, val mappings: List<RangeMapping>)

In [4]:
fun getSeeds(lines: List<String>): List<Entity> {
    val regex = "^seeds:\\s+(.*)$".toRegex()
    val seedIds = lines.firstNotNullOf { line -> regex.matchEntire(line) }.groupValues[1].split(" ")
    return seedIds.map { Entity("seed", it.toLong()) }
}

In [5]:
fun parse(input: List<String>): Map<String, TypeMapping> {
    val sectionRegex = "^(.*?)-to-(.*?) map:".toRegex()
    return input
        .drop(2)
        .fold(emptyList<Pair<String, TypeMapping>>()) { pairs, line ->
            val matchResult = sectionRegex.matchEntire(line)
            if (matchResult != null) {
                val srcType = matchResult.groupValues[1]
                val destType = matchResult.groupValues[2]
                val newPair = srcType to TypeMapping(srcType, destType, emptyList())
                pairs + newPair
            } else if (line.isBlank()) {
                pairs
            } else {
                val (type, typeMapping) = pairs.last()
                val nums = line.split(" ")
                check(nums.size == 3)
                val (destRangeStart, srcRangeStart, rangeLength) = nums.map(String::toLong)
                val newRangeMapping = RangeMapping(
                    srcRange = srcRangeStart..srcRangeStart + rangeLength - 1,
                    offset = destRangeStart - srcRangeStart
                )
                val newPair = type to typeMapping.copy(mappings = typeMapping.mappings + newRangeMapping)
                pairs.dropLast(1) + newPair
            }
        }
        .toMap()
}

fun getFinalId(entity: Entity, typeMappings: Map<String, TypeMapping>): Entity =
    generateSequence(entity) { nextEntity ->
        typeMappings[nextEntity.type]
            ?.let { typeMapping ->
                val rangeMapping = typeMapping.mappings.firstOrNull { it.srcRange.contains(nextEntity.id) }
                Entity(typeMapping.destType, nextEntity.id + (rangeMapping?.offset ?: 0))
            }
    }.last()

In [6]:
fun findNearestLocation(input: List<String>): Long {
    val seeds = getSeeds(input)
    val map = parse(input)
    return seeds.asSequence().map { seed -> getFinalId(seed, map) }.map(Entity::id).min()
}

In [7]:
findNearestLocation(actualInput.lines())

//173706076

173706076

# Part 2

In [8]:
fun getSeedsAsRange(lines: List<String>): Sequence<LongRange> {
    val regex = "^seeds:\\s+(.*)$".toRegex()
    val seedNums = lines.firstNotNullOf { line -> regex.matchEntire(line) }.groupValues[1].split(" ")
    return seedNums
        .asSequence()
        .chunked(2)
        .map { (startNumStr, lengthStr) ->
            val startNum = startNumStr.toLong()
            val length = lengthStr.toLong()
            val longRange = startNum until startNum + length
            println(longRange.count())
            longRange
        }
}

In [9]:
import java.util.Spliterators
import java.util.Spliterator
import java.util.stream.Stream
import java.util.stream.StreamSupport

fun <T> Sequence<T>.asStream(): Stream<T> =
    StreamSupport.stream(
        { Spliterators.spliteratorUnknownSize(iterator(), Spliterator.ORDERED) },
        Spliterator.ORDERED,
        false
    )

fun findNearestLocationPart2(input: List<String>): Long {
    val seeds = getSeedsAsRange(input)
    val map = parse(input)
    return seeds
        .asStream()
        .parallel()
        .map { idRange ->
            idRange
                .asSequence()
                .asStream()
                .parallel()
                .also { println("ID range $it") }
                .map { seedId ->
                    getFinalId(Entity("seed", seedId), map).id
                }
                .min(Comparator { a, b -> a.compareTo(b) })
                .get()
        }
        .min(Comparator { a, b -> a.compareTo(b) })
        .get()
}

In [10]:
findNearestLocationPart2(actualInput.lines())

// 11611182


56936593
267019426
252409474
92561087
24162055
63600948
323477533
54368890
475537300
427659734
ID range java.util.stream.ReferencePipeline$Head@178a628e
ID range java.util.stream.ReferencePipeline$Head@6563f88c
ID range java.util.stream.ReferencePipeline$Head@3436fbd1
ID range java.util.stream.ReferencePipeline$Head@24a4d763
ID range java.util.stream.ReferencePipeline$Head@4e763388
ID range java.util.stream.ReferencePipeline$Head@342b0552
ID range java.util.stream.ReferencePipeline$Head@37408875
ID range java.util.stream.ReferencePipeline$Head@355d5e31
ID range java.util.stream.ReferencePipeline$Head@4a48c161
ID range java.util.stream.ReferencePipeline$Head@26a54cef


11611182